Dual energy carbonate CT
========================

test


In [1]:
import ipyparallel as ipp
cluster = ipp.Cluster(engines="mpi", n=5)
rc = cluster.start_and_connect_sync()
rc[:].activate() # enable magics

Starting 5 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/5 [00:00<?, ?engine/s]

In [ ]:
%%px --block --group-outputs=type

import matplotlib.pyplot as plt
from IPython.display import display
import rockverse as rv

dectgroup = rv.dualenergyct.create_group(
    store='E:/path/to/dual_energy_ct/C04B21',
    overwrite=True)

dectgroup.copy_image(
    image=rv.open('E:/path/to/imported/dual_energy_carbonate/C04B21Raw100keV'),
    path='lowECT',
    overwrite=True)

dectgroup.copy_image(
    image=rv.open('E:/path/to/imported/dual_energy_carbonate/C04B21Raw140keV'),
    path='highECT',
    overwrite=True)

Out[1:2]: <rockverse.dualenergyct.DualEnergyCTGroup at 0x1df0c5dad50>

Out[0:2]: <rockverse.dualenergyct.DualEnergyCTGroup at 0x2afc045a840>

Out[4:2]: <rockverse.dualenergyct.DualEnergyCTGroup at 0x23c2d056cc0>

Out[3:2]: <rockverse.dualenergyct.DualEnergyCTGroup at 0x2b302758890>

Out[2:2]: <rockverse.dualenergyct.DualEnergyCTGroup at 0x2cd56daf0e0>

In [ ]:
%%px --block --group-outputs=type
lowE_viewer = rv.OrthogonalViewer(image=dectgroup.lowECT)
highE_viewer = rv.OrthogonalViewer(image=dectgroup.highECT)

if rv.config['MPI']['mpi_rank'] != 0:
    plt.close(lowE_viewer.figure)
    plt.close(highE_viewer.figure)


In [ ]:
%%px --block --group-outputs=type

#Test with regions
highE_viewer.figure.set_size_inches(10, 10)

highE_viewer.mask_color = 'gold'
highE_viewer.mask_alpha = 0.5

air_region = rv.region.Cylinder(p=(126, 20, 461), v=(0, 0, 1), r=10, l=750)
highE_viewer.region = air_region

highE_viewer.ax_histogram.set_xlim(-1200, 3500)

if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)


In [ ]:
%%px --block --group-outputs=type

#Test with regions
water_region = rv.region.Cylinder(p=(55, 176.2, 461), v=(0, 0, 1), r=10, l=900)
highE_viewer.region = water_region
highE_viewer.ref_point = water_region.p
highE_viewer.ax_histogram.set_xlim(-1200, 3500)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type

#Test with regions
teflon_region = rv.region.Cylinder(p=(124.7, 228, 461), v=(0, 0, 1), r=6.5, l=900)
highE_viewer.region = teflon_region
highE_viewer.ref_point = teflon_region.p
highE_viewer.ax_histogram.set_xlim(-1200, 3500)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type

#Test with regions
silica_region = rv.region.Cylinder(p=(196, 179, 461), v=(0, 0, 1), r=9, l=900)
highE_viewer.region = silica_region
highE_viewer.ref_point = silica_region.p
highE_viewer.ax_histogram.set_xlim(-1200, 3500)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type

#Test with regions
rock_region = rv.region.Cylinder(p=(126, 114, 461), v=(0, 0, 1), r=63, l=875)
highE_viewer.region = rock_region
highE_viewer.ref_point = rock_region.p
highE_viewer.ax_histogram.set_xlim(-1200, 3500)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type

#DECT does not take regions, combine in a single mask
dectgroup.create_mask(fill_value=True, overwrite=True)
dectgroup.mask.math(value=False, op='set', region=air_region)
dectgroup.mask.math(value=False, op='set', region=water_region)
dectgroup.mask.math(value=False, op='set', region=teflon_region)
dectgroup.mask.math(value=False, op='set', region=silica_region)
dectgroup.mask.math(value=False, op='set', region=rock_region)
highE_viewer.region = None
highE_viewer.mask = dectgroup.mask
highE_viewer.ax_histogram.set_xlim(-1200, 3500)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type

dectgroup.create_segmentation(fill_value=0, overwrite=True)
dectgroup.segmentation.math(value=1, op='set', region=air_region)
dectgroup.segmentation.math(value=2, op='set', region=water_region)
dectgroup.segmentation.math(value=3, op='set', region=teflon_region)
dectgroup.segmentation.math(value=4, op='set', region=silica_region)
dectgroup.segmentation.math(value=5, op='set', region=rock_region)


dectgroup.mask.math(value=True, op='set', region=rv.region.Cylinder(r=10000, p=rock_region.p, v=(0,0,1), l=10, region='outside'))

highE_viewer.segmentation = dectgroup.segmentation
highE_viewer.mask_color = 'k'
highE_viewer.mask_alpha = 0.75
highE_viewer.ax_histogram.set_xlim(-1200, 3500)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type

highE_viewer.histogram_bins=2**10
highE_viewer.update_image_dict(clim=(-1200, 3000))
highE_viewer.ax_histogram.set_xlim(-1200, 3000)

highE_viewer.ax_histogram.legend(
    [
        highE_viewer.histogram_lines['full'],
        highE_viewer.histogram_lines['1'],
        highE_viewer.histogram_lines['2'],
        highE_viewer.histogram_lines['3'],
        highE_viewer.histogram_lines['4'],
        highE_viewer.histogram_lines['5'],
    ], [
        'Full',
        'Air',
        'Water',
        'Teflon',
        'SiO2',
        'Rock'
    ]
)
if rv.config['MPI']['mpi_rank'] == 0:
    display(highE_viewer.figure)

In [ ]:
%%px --block --group-outputs=type


dectgroup.histogram_bins = 2**10

dectgroup.calibration_material0['description'] = 'Air'
dectgroup.calibration_material0['segmentation_phase'] = 1
dectgroup.calibration_material0['lowE_gaussian_center_bounds'] = [-1050, -950]
dectgroup.calibration_material0['highE_gaussian_center_bounds'] = [-1050, -950]

dectgroup.calibration_material1['description'] = 'Water'
dectgroup.calibration_material1['segmentation_phase'] = 2
dectgroup.calibration_material1['composition'] = {'H': 2, 'O': 1}
dectgroup.calibration_material1['bulk_density'] = 1
dectgroup.calibration_material1['lowE_gaussian_center_bounds'] = [-100, 100]
dectgroup.calibration_material1['highE_gaussian_center_bounds'] = [-100, 100]

dectgroup.calibration_material2['description'] = 'SiO2'
dectgroup.calibration_material2['segmentation_phase'] = 3
dectgroup.calibration_material2['composition'] = {'Si': 1, 'O': 2}
dectgroup.calibration_material2['bulk_density'] = 2.2
dectgroup.calibration_material2['lowE_gaussian_center_bounds'] = [1550, 1700]
dectgroup.calibration_material2['highE_gaussian_center_bounds'] = [1300, 1500]

dectgroup.calibration_material3['description'] = 'Teflon'
dectgroup.calibration_material3['segmentation_phase'] = 4
dectgroup.calibration_material3['composition'] = {'C': 2, 'F': 4}
dectgroup.calibration_material3['bulk_density'] = 2.2
dectgroup.calibration_material3['lowE_gaussian_center_bounds'] = [1000, 1200]
dectgroup.calibration_material3['highE_gaussian_center_bounds'] = [1000, 1100]

dectgroup.preprocess()


#MELHORAR O OUPTUT SEM LINHAS NO TQDM

FAZER O PLOT

In [ ]:
%%px --block --group-outputs=type
dectgroup.run()